Libraries

In [ ]:
# Libraries
import numpy as np
import sys
import os
import json
from pathlib import Path
import re
import litellm 
from pydantic import BaseModel
import pandas as pd

# add path to the dataset entities
sys.path.append(os.path.abspath("../0. Helpers"))
sys.path.append(os.path.abspath("../2. Data Processing/_dataset_entities"))

from datasetProcessing import Entity, recursive_fix
from reflection_helpers import get_token_context_include, get_token_context_exclude, get_entity_context, get_entity_inner_boundary

Settings

In [ ]:
# potential_tokens_folder = "paper"
# prob_threshold = 0.95

potential_tokens_folder = "adapted"

Topic

In [ ]:
all_configs = {
    "ai": 10,
    "literature": 10,
    "music": 10,
    "politics": 20,
    "science": 20,
    "multinerd_en": 20,
    "multinerd_pt": 20,
    "ener": 20,
    "lener": 20,
    "neuralshift": 20
}

Process functions

In [ ]:
def process_token(topic, token, boundary, entity_str):

    # Search for examples where token is inside
    classification_path = Path(f"classification/{topic}/train/data")

    examples = []

    # classification files
    for file in classification_path.glob("*.json"):

        with open(file, mode='r', encoding="utf-8") as f:
            content = f.read()
            
        content = re.sub(r',\s*$', '', content)
        data = json.loads(content)

        true_entities = data["true_entities"]
        entity_tokens = data["classification"]["entity"]
        context_tokens = data["classification"]["context"]

        # if boundary is inside, check context tokens
        if boundary == "inside":
            if token in context_tokens:
                for entity in true_entities:
                    # get context
                    context = get_entity_context(data["sentence"], entity["span"], context_length = 4)
                    if context and token in context:
                        
                        demonstration = {
                            "sentence": f"...{' '.join(context)}...",
                            "entity": entity["span"]
                        }
                        examples.append(demonstration)

        # if boundary is outside, check entity tokens
        elif boundary == "outside":
            if token in entity_tokens:
                for entity in true_entities:
                    # check if token is in this entity span
                    if token in entity["span"]:
                        context = get_entity_context(data["sentence"], entity["span"], context_length = 4)
                        if context:
                            demonstration = {
                                "sentence": f"...{' '.join(context)}...",
                                "entity": entity["span"]
                            }
                            examples.append(demonstration)

        else:
             raise ValueError("Boundary must be either 'inside' or 'outside'")

    token_json = {
        "boundary": boundary,
        "token": token,
        "entity_str": entity_str,
        "examples": examples,
    }
    
    return token_json

In [ ]:
def identify_boundary(sentence, llm_entities, probs_dict):

    outer_tokens = []
    inner_tokens = []

    for entity in llm_entities:

        entity_str = f"{{'span': '{entity['span']}', 'entity': '{entity['entity']}'}}"

        # process outer context
        outer_context = get_entity_context(sentence, entity["span"], context_length = 1, include_entity = False)

        if not outer_context:
            print()
            print("❌-------------")
            print("sentence: ",   sentence)
            print("entity:   ", entity["span"])
            print("❌-------------")
            print()
            continue
            

        # check tokens that are likely entity 
        for token in outer_context:
            if token in probs_dict:

                # paper vs adapted!
                if potential_tokens_folder == "adapted":
                    high_prob = probs_dict[token]["prob_e"] == 1
                elif potential_tokens_folder == "paper" and prob_threshold:
                    high_prob = probs_dict[token]["prob_e"] >= prob_threshold
                else:
                    raise ValueError("Invalid potential_tokens_folder")
                
                if high_prob:
                    
                    # check if token is not inside a predicted entity
                    if all(token.lower() not in e["span"].lower() for e in llm_entities) and token not in outer_tokens:
                        outer_tokens.append((token, entity_str))

        # process inner context
        inner_context = get_entity_inner_boundary(entity["span"], context_length = 1)

        # check tokens that are likely context 
        for token in inner_context:
            if token in probs_dict and probs_dict[token]["prob_c"] == 1:
                inner_tokens.append((token, entity_str))

    return inner_tokens, outer_tokens


In [ ]:
def process_instance(topic, file_path, probs_dict):
    
    with open(file_path, mode='r', encoding="utf-8") as f:
        content = f.read()

    if not content.strip():
        print(f"🗑️ Empty file detected: {file_path}")
        return None

    # Fix JSON extra comma
    content = re.sub(r',\s*$', '', content)
    data = json.loads(content)

    # Apply encoding fix
    data = recursive_fix(data)  

    # extract entities
    # true_entities = data.get("true_entities", [])
    llm_entities = data["entities"]
    sentence = data["sentence"]

    inside_boundary, outside_boundary = identify_boundary(sentence, llm_entities, probs_dict)
    inside_boundary_info, outside_boundary_info = [], []

    if inside_boundary or outside_boundary:
        print(f"Found {len(inside_boundary)} inside boundary tokens and {len(outside_boundary)} outside boundary tokens in {file_path}")
        print("Sentence: ", sentence)
        print(inside_boundary)
        print(outside_boundary)
        print("\n---\n")

        for token, entity_str in inside_boundary:
            token_with_examples = process_token(topic, token, "inside", entity_str)

            # only keep tokens with more than 1 positive example
            if len(token_with_examples["examples"]) > 1:
                inside_boundary_info.append(token_with_examples)

        for token, entity_str in outside_boundary:
            token_with_examples = process_token(topic, token, "outside", entity_str)

            # only keep tokens with more than 1 positive example
            if len(token_with_examples["examples"]) > 1:
                outside_boundary_info.append(token_with_examples)

        if inside_boundary_info or outside_boundary_info:
            instance_output_json = {
                "sentence": sentence,
                "inside_boundary_tokens": inside_boundary_info,
                "outside_boundary_tokens": outside_boundary_info
            }

            # save token to a file
            output_path = f"error_reflection/boundary/{topic}/{potential_tokens_folder}/{Path(file_path).stem}.json"
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(json.dumps(instance_output_json, ensure_ascii=False, indent=4))

    return inside_boundary_info, outside_boundary_info

Run for all configs

In [ ]:
boundary_summary = {}

for topic, n in all_configs.items():

    print(f"Processing topic: {topic} with top {n} demos")

    best_results_folder = f"results/demo_type/{topic}/in_context_top{n}"

    # ensure folder exists
    os.makedirs(f"error_reflection/boundary/{topic}/{potential_tokens_folder}", exist_ok=True)

    # load probs dict
    probs_path = f"classification/{topic}/train/_probs.json"

    # read file
    with open(probs_path, "r", encoding="utf-8") as f:
        probs_dict = json.load(f)

    # Process all instances in the folder
    instances = 0
    instances_with_inside_boundary = 0
    instances_with_outside_boundary = 0

    total_inside_boundary_tokens = 0
    total_outside_boundary_tokens = 0

    for file_path in Path(best_results_folder).glob("*.json"):
        instances += 1

        inside_boundary_tokens, outside_boundary_tokens = process_instance(topic, file_path, probs_dict)
        
        if inside_boundary_tokens:
            instances_with_inside_boundary += 1
            total_inside_boundary_tokens += len(inside_boundary_tokens)

        if outside_boundary_tokens:
            instances_with_outside_boundary += 1
            total_outside_boundary_tokens += len(outside_boundary_tokens)

    boundary_summary[topic] = {
        "instances": instances,
        "instances_with_inside_boundary": instances_with_inside_boundary,
        "instances_with_inside_boundary_percent": instances_with_inside_boundary / instances * 100,
        "total_inside_boundary_tokens": total_inside_boundary_tokens,
        "instances_with_outside_boundary": instances_with_outside_boundary,
        "instances_with_outside_boundary_percent": instances_with_outside_boundary / instances * 100,
        "total_outside_boundary_tokens": total_outside_boundary_tokens,
    }
    

In [ ]:
for topic, summary in boundary_summary.items():
    print(f"Topic: {topic}")
    print(f"Instances with inside boundary: {summary['instances_with_inside_boundary']} ({summary['instances_with_inside_boundary_percent']:.1f}%)")
    print(f"Total inside boundary tokens: {summary['total_inside_boundary_tokens']}")

    print(f"Instances with outside boundary: {summary['instances_with_outside_boundary']} ({summary['instances_with_outside_boundary_percent']:.1f}%)")
    print(f"Total outside boundary tokens: {summary['total_outside_boundary_tokens']}")

    print("\n---")